<a href="https://colab.research.google.com/github/YuhengWillZhao/Econ515_William/blob/main/515_William.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Revised full code: display-only (no file writes)
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.ar_model import AutoReg
from scipy.stats import chi2
from datetime import datetime
import math
import seaborn as sns

# ----------------- Utility: robust CSV loader -----------------
def load_price_csv(path):
    tmp = pd.read_csv(path)
    # normalize column names
    cols = [c.strip() for c in tmp.columns]
    tmp.columns = cols
    lowcols = [c.lower() for c in cols]
    # date column
    if 'date' in lowcols:
        date_col = cols[lowcols.index('date')]
    elif 'data' in lowcols:
        date_col = cols[lowcols.index('data')]
    else:
        date_col = cols[0]
    # price column
    if 'price' in lowcols:
        price_col = cols[lowcols.index('price')]
    else:
        price_col = [c for c in cols if c != date_col][0]
    df = tmp[[date_col, price_col]].rename(columns={date_col:'date', price_col:'price'})
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date').sort_index()
    return df

# ----------------- Load data (paths per your upload) -----------------
spot = load_price_csv('spot.csv')
future = load_price_csv('future.csv')
tbill = load_price_csv('TB3MS (1).csv')

# ----------------- Align to sample and business days -----------------
start_date = pd.to_datetime('2020-01-01')
end_date = pd.to_datetime('2024-03-31')
idx = pd.date_range(start=start_date, end=end_date, freq='B')


spot = spot.reindex(idx).rename(columns={'price':'spot'})
future = future.reindex(idx).rename(columns={'price':'future'})
tbill = tbill.reindex(idx).rename(columns={'price':'tbill'})

# forward-fill then backfill
spot['spot'] = spot['spot'].ffill().bfill()
future['future'] = future['future'].ffill().bfill()
tbill['tbill'] = tbill['tbill'].ffill().bfill()

# combine
df = pd.concat([spot['spot'], future['future'], tbill['tbill']], axis=1).dropna()

# logs, diffs, and Rt
df['log_spot'] = np.log(df['spot'])
df['log_future'] = np.log(df['future'])
df['dlog_spot'] = df['log_spot'].diff()
df['dlog_future'] = df['log_future'].diff()
df['Rt'] = df['tbill'] / 1200.0
df = df.dropna()

# ----------------- Display: head -----------------
print("Prepared data (head):")
display(df[['spot','future','tbill','Rt']].head())

# ----------------- Unit root tests (ADF, KPSS) -----------------
def do_ur_tests(series):
    out = {}
    s = series.dropna()
    try:
        adf = adfuller(s, autolag='AIC')
        out['ADF_stat'] = adf[0]; out['ADF_pvalue'] = adf[1]
    except Exception:
        out['ADF_stat'] = np.nan; out['ADF_pvalue'] = np.nan
    try:
        kps = kpss(s, regression='c', nlags='auto')
        out['KPSS_stat'] = kps[0]; out['KPSS_pvalue'] = kps[1]
    except Exception:
        out['KPSS_stat'] = np.nan; out['KPSS_pvalue'] = np.nan
    return out

ur_results = {
    'log_spot': do_ur_tests(df['log_spot']),
    'log_future': do_ur_tests(df['log_future']),
    'dlog_spot': do_ur_tests(df['dlog_spot']),
    'dlog_future': do_ur_tests(df['dlog_future'])
}
ur_table = pd.DataFrame(ur_results).T
print("\nUnit-root test summary (ADF, KPSS):")
display(ur_table)

# ----------------- Johansen cointegration (trace) -----------------
print("\nJohansen cointegration (trace) baseline:")
jres = coint_johansen(df[['log_spot','log_future']], det_order=0, k_ar_diff=1)
jtab = []
for i,(lr,val) in enumerate(zip(jres.lr1, jres.eig)):
    jtab.append({'r<=%d' % i: i, 'trace_stat': lr, 'eigenvalue': val,
                 'crit_90': jres.cvt[i,0], 'crit_95': jres.cvt[i,1], 'crit_99': jres.cvt[i,2]})
display(pd.DataFrame(jtab))

# ----------------- Full-sample OLS cointegrating regression -----------------
X_full = sm.add_constant(df['log_future'])
ols_full = sm.OLS(df['log_spot'], X_full).fit()
ols_full_hac = ols_full.get_robustcov_results(cov_type='HAC', maxlags=12)
full_table = pd.DataFrame({
    'estimate': ols_full.params,
    'hac_se': ols_full_hac.bse,
    't': ols_full_hac.tvalues,
    'pvalue': ols_full_hac.pvalues
})
print("\nFull-sample OLS cointegrating regression (HAC SEs):")
display(full_table.T)

# residuals
df['coint_resid'] = ols_full.resid.values

# ----------------- Bai–Perron style segmentation (dynamic programming) -----------------
T = len(df)
trim = int(np.floor(0.15 * T))   # 15% trimming
max_breaks = 5
y = df['log_spot'].values
X = sm.add_constant(df['log_future']).values

# SSR precompute
min_len = 4
SSR = np.full((T+1, T+1), np.inf)
for i in range(0, T):
    for j in range(i+min_len, T+1):
        Xi = X[i:j,:]; yi = y[i:j]
        try:
            b = np.linalg.lstsq(Xi, yi, rcond=None)[0]
            resid = yi - Xi.dot(b)
            SSR[i,j] = float(np.sum(resid**2))
        except Exception:
            SSR[i,j] = np.inf

# dynamic programming for partitions
def best_partition(m):
    K = m+1
    dp = np.full((K+1, T+1), np.inf)
    prev = -np.ones((K+1, T+1), dtype=int)
    dp[0,0] = 0.0
    for k in range(1, K+1):
        jmin = k*trim
        jmax = T - (K-k)*trim
        for j in range(jmin, jmax+1):
            best = np.inf; arg = -1
            i_min = (k-1)*trim
            i_max = j - trim
            for i in range(i_min, i_max+1):
                s = SSR[i,j]
                if np.isinf(s): continue
                cand = dp[k-1,i] + s
                if cand < best:
                    best = cand; arg = i
            dp[k,j] = best; prev[k,j] = arg
    if np.isinf(dp[K,T]):
        return None, np.inf
    # reconstruct
    ends = []; j=T; k=K
    while k>0:
        i = prev[k,j]
        ends.append(j)
        j = i; k -= 1
    ends.reverse()
    break_positions = ends[:-1]
    return break_positions, dp[K,T]

results = {}
for m in range(0, max_breaks+1):
    bpos, ssr = best_partition(m)
    results[m] = {'break_positions': bpos, 'total_ssr': ssr}

# show SSR vs m and break dates
ssr_list = []
breaks_list = []
for m in range(0, max_breaks+1):
    bp = results[m]['break_positions']
    ssr_list.append(results[m]['total_ssr'])
    if bp:
        breaks_list.append([df.index[p-1].strftime('%Y-%m-%d') for p in bp])
    else:
        breaks_list.append([])

print("\nSegmentation results (SSR by m) and selected partition candidate breaks:")
seg_df = pd.DataFrame({'m': list(range(0, max_breaks+1)), 'SSR': ssr_list, 'breaks': breaks_list})
display(seg_df)

# plot SSR vs m
plt.figure(figsize=(6,4))
plt.plot(seg_df['m'], seg_df['SSR'], marker='o')
plt.xlabel('Number of breaks (m)')
plt.ylabel('Total SSR (minimized)')
plt.title('SSR minimized vs number of breaks (Bai–Perron DP)')
plt.grid(True)
plt.show()

# ----------------- Compute raw Wald for chosen partitions (approx sup-Wald) -----------------
def estimate_regimes_and_wald(break_positions):
    starts = [0] + (break_positions if break_positions else [])
    ends = (break_positions if break_positions else []) + [T]
    params = []
    covs = []
    SSR_seg = 0.0
    for s,e in zip(starts, ends):
        Xi = X[s:e,:]; yi = y[s:e]
        beta = np.linalg.lstsq(Xi, yi, rcond=None)[0]
        params.append(beta)
        resid = yi - Xi.dot(beta)
        SSR_seg += float(np.sum(resid**2))
        # HAC cov via statsmodels on the sub-sample
        sub = df.iloc[s:e]
        ols_sub = sm.OLS(sub['log_spot'], sm.add_constant(sub['log_future'])).fit()
        robust = ols_sub.get_robustcov_results(cov_type='HAC', maxlags=12)
        covs.append(np.asarray(robust.cov_params()))
    pooled_beta = np.linalg.lstsq(X, y, rcond=None)[0]
    # stacked differences
    stacked = np.concatenate([p - pooled_beta for p in params], axis=0)
    # block-diagonal covariance
    k = params[0].shape[0]; m = len(params)
    bigcov = np.zeros((m*k, m*k))
    for i in range(m):
        bigcov[i*k:(i+1)*k, i*k:(i+1)*k] = covs[i]
    try:
        inv_bigcov = np.linalg.pinv(bigcov)
        wald = float(stacked.T.dot(inv_bigcov.dot(stacked)))
    except Exception:
        wald = np.nan
    return {'wald': wald, 'SSRp': float(np.sum((y - X.dot(pooled_beta))**2)), 'SSRseg': SSR_seg, 'params': params, 'covs': covs}

sup_wald = {}
for m in range(1, max_breaks+1):
    bp = results[m]['break_positions']
    if bp is None:
        sup_wald[m] = {'wald': np.nan, 'break_positions': bp}
    else:
        est = estimate_regimes_and_wald(bp)
        sup_wald[m] = {'wald': est['wald'], 'break_positions': bp, 'SSRseg': est['SSRseg'], 'SSRp': est['SSRp']}

print("\nRaw Wald stats for candidate partitions (per m):")
rows = []
for m in range(1, max_breaks+1):
    bp = sup_wald[m]['break_positions']
    dates = [df.index[p-1].strftime('%Y-%m-%d') for p in bp] if bp else []
    rows.append({'m': m, 'raw_wald': sup_wald[m]['wald'], 'breaks': dates})
display(pd.DataFrame(rows))

# choose m_hat by UDmax (max raw_wald)
valid = {m: sup_wald[m]['wald'] for m in sup_wald if not np.isnan(sup_wald[m]['wald'])}
if len(valid)==0:
    m_hat = 0
else:
    m_hat = max(valid, key=valid.get)
print("\nSelected number of breaks m_hat (UDmax raw-wald):", m_hat)
chosen_bp = sup_wald[m_hat]['break_positions'] if m_hat>0 else []
chosen_dates = [df.index[p-1].strftime('%Y-%m-%d') for p in chosen_bp] if chosen_bp else []
print("Chosen break dates:", chosen_dates)

# ----------------- Bootstrap empirical p-value for sup-Wald (residual bootstrap) -----------------
def bootstrap_sup_wald(break_positions, nboot=400, seed=1234):
    rng = np.random.default_rng(seed)
    pooled_beta = np.linalg.lstsq(X, y, rcond=None)[0]
    resid_pool = y - X.dot(pooled_beta)
    obs_wald = sup_wald[m_hat]['wald']
    w_star = []
    for b in range(nboot):
        e_star = rng.choice(resid_pool, size=T, replace=True)
        y_star = X.dot(pooled_beta) + e_star
        # compute regime betas & covs on y_star for the chosen partition
        # small speed-up: compute OLS per regime and homoskedastic cov approx
        starts = [0] + (break_positions if break_positions else [])
        ends = (break_positions if break_positions else []) + [T]
        params_star = []
        bigcov_star = np.zeros((len(starts)*2, len(starts)*2))
        stacked_diff = []
        for i,(s,e) in enumerate(zip(starts, ends)):
            Xi = X[s:e,:]; yi_star = y_star[s:e]
            beta_star = np.linalg.lstsq(Xi, yi_star, rcond=None)[0]
            params_star.append(beta_star)
            resid_seg = yi_star - Xi.dot(beta_star)
            sigma2 = np.var(resid_seg, ddof=Xi.shape[1])
            cov_beta = sigma2 * np.linalg.pinv(Xi.T.dot(Xi))
            bigcov_star[i*2:(i+1)*2, i*2:(i+1)*2] = cov_beta
            stacked_diff.append(beta_star - pooled_beta)
        stacked = np.concatenate(stacked_diff, axis=0)
        try:
            inv_bigcov = np.linalg.pinv(bigcov_star)
            w = float(stacked.T.dot(inv_bigcov.dot(stacked)))
        except Exception:
            w = np.nan
        w_star.append(w)
    w_star = np.array(w_star)
    pval = np.mean(w_star >= obs_wald)
    return {'obs_wald': obs_wald, 'wald_star': w_star, 'pvalue': pval}

if m_hat > 0:
    print("\nRunning residual bootstrap (this will take ~1-2 minutes depending on nboot)...")
    boot = bootstrap_sup_wald(chosen_bp, nboot=400, seed=2025)
    print("Bootstrap empirical p-value for sup-Wald (chosen partition):", boot['pvalue'])
    # histogram
    plt.figure(figsize=(7,4))
    sns.histplot(boot['wald_star'], bins=40, kde=False)
    plt.axvline(boot['obs_wald'], color='red', linestyle='--', label=f'Observed Wald={boot["obs_wald"]:.2f}')
    plt.xlabel('Bootstrap Wald*')
    plt.ylabel('Frequency')
    plt.title('Bootstrap distribution of Wald* (residual bootstrap)\nObserved Wald marked in red')
    plt.legend()
    plt.show()
else:
    print("\nNo breaks selected (m_hat=0) — skipping bootstrap.")

# ----------------- Display cointegration residuals with chosen breaks -----------------
plt.figure(figsize=(12,4))
plt.plot(df.index, df['coint_resid'], label='cointegration residuals')
for b in chosen_bp:
    plt.axvline(df.index[b-1], color='red', linestyle='--', linewidth=1)
plt.title('Cointegration residuals with chosen breakpoints (vertical dashed lines)')
plt.xlabel('Date')
plt.legend()
plt.show()

# ----------------- Regime-wise OLS estimates (final) -----------------
regimes = []
start_idx = 0
parts = (chosen_bp if chosen_bp else []) + [T]
for end in parts:
    sub = df.iloc[start_idx:end]
    Xsub = sm.add_constant(sub['log_future'])
    ols_sub = sm.OLS(sub['log_spot'], Xsub).fit()
    ols_sub_hac = ols_sub.get_robustcov_results(cov_type='HAC', maxlags=12)
    regimes.append({
        'start': sub.index[0].strftime('%Y-%m-%d'),
        'end': sub.index[-1].strftime('%Y-%m-%d'),
        'coef_const': float(ols_sub.params['const']),
        'coef_beta': float(ols_sub.params['log_future']),
        'se_const': float(ols_sub_hac.bse[0]),
        'se_beta': float(ols_sub_hac.bse[1]),
        'nobs': len(sub)
    })
    start_idx = end

reg_df = pd.DataFrame(regimes)
print("\nRegime-wise cointegration estimates (HAC SEs):")
display(reg_df)

# plot coefficient bar chart with error bars
plt.figure(figsize=(8,4))
x_positions = np.arange(len(reg_df))
plt.errorbar(x_positions, reg_df['coef_beta'], yerr=1.96*reg_df['se_beta'], fmt='o', capsize=5)
plt.xticks(x_positions, [f"{r['start']}→{r['end']}" for _,r in reg_df.iterrows()], rotation=30, ha='right')
plt.ylabel('Beta (cointegrating slope)')
plt.title('Regime-wise cointegration slope estimates (95% CI)')
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

# ----------------- AK-like LM approximation and diagnostic -----------------
resid = df['coint_resid'].values
best_p = 1; best_aic = 1e12; best_model = None
for p in range(1,7):
    try:
        m_ar = AutoReg(resid, lags=p, old_names=False).fit()
        if hasattr(m_ar,'aic') and m_ar.aic < best_aic:
            best_aic = m_ar.aic; best_p = p; best_model = m_ar
    except Exception:
        pass

if best_model is not None:
    if hasattr(best_model, 'rsquared'):
        R2 = best_model.rsquared
    else:
        SSR_ar = np.sum(best_model.resid**2)
        var_resid = np.sum((resid - resid.mean())**2)
        R2 = 1 - SSR_ar / var_resid
    LM_stat = len(resid) * R2
    pval_LM = 1 - chi2.cdf(LM_stat, df=best_p)
else:
    LM_stat = np.nan; pval_LM = np.nan

print("\nAK-like LM approximation (informative):")
print(f" Selected AR lag p = {best_p}")
print(f" LM_stat ≈ {LM_stat:.4f}, approx p-value (chi2 df={best_p}) ≈ {pval_LM:.6g}")

# quick ACF plot of residuals for diagnostics
from statsmodels.graphics.tsaplots import plot_acf
plt.figure(figsize=(8,3))
plot_acf(resid, lags=30, alpha=0.05)
plt.title('ACF of cointegration residuals (diagnostic for AK-LM)')
plt.tight_layout()
plt.show()


# ----------------- Final summary -----------------
# ----------------- Final summary -----------------
print("\n--- Summary (presentation-ready) ---")

# Retrieve HAC SEs correctly by position
beta_se = float(ols_full_hac.bse[1])   # 2nd element corresponds to 'log_future'
const_se = float(ols_full_hac.bse[0])

print(f" Sample: {start_date.date()} to {end_date.date()}, {T} business-day obs used (after trimming NAs).")
print(f" Full-sample cointegrating slope (beta): {ols_full.params['log_future']:.6f} (HAC se {beta_se:.6f})")
print(f" Constant term: {ols_full.params['const']:.6f} (HAC se {const_se:.6f})")
print(f" Selected number of breaks (m_hat): {m_hat}; break dates: {chosen_dates}")
if m_hat > 0:
    print(f" Bootstrap empirical p-value for sup-Wald (chosen partition): {pval_boot:.4f}")
print(f" AK-like LM approx stat: {LM_stat:.4f}, approx p-value: {pval_LM:.6g}")


FileNotFoundError: [Errno 2] No such file or directory: 'spot.csv'